In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from ortools.sat.python import cp_model

# Instance Class

In [2]:
class Instance:
    O: int  # number of requests
    I: int  # number of items
    A: int  # number of corridors

    u_oi: np.ndarray  # quantity of each item for each request
    u_ai = np.ndarray  # quantity of each item inside each corridor

    LB: int  # lower bound on the wave size
    UB: int  # upper bound on the wave size

    def __init__(self, input_file):
        with open(input_file, 'r') as file:
            self.O, self.I, self.A = map(int, file.readline().split())

            # Read orders matrix:
            self.u_oi = np.zeros((self.O, self.I), dtype=int)
            for i in range(self.O):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_oi[i, indices] = qtys

            # Read corridors matrix:
            self.u_ai = np.zeros((self.A, self.I), dtype=int)
            for i in range(self.A):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_ai[i, indices] = qtys

            # Read lower and upper bounds for wave size:
            self.LB, self.UB = map(int, file.readline().split())

In [3]:
 inst = Instance('datasets/a/instance_0005.txt')

# Linearized model

Based on a linearization [trick](https://lpsolve.sourceforge.net/5.5/ratio.htm).

In [4]:
%%time
# Create a new model
n = gp.Model('meli-linearized')

## Variables:
θ = n.addMVar(inst.O, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='θ')  # wave selected requests times d
α = n.addMVar(inst.A, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='α')  # wave visited corridors times d
d = n.addVar(vtype=GRB.CONTINUOUS, lb=1 / inst.A, ub=1, name='d')  # denominator of the objective cost function = 1/wave_corridors_
n.update()

## Restrictions:
wave_size_ = (inst.u_oi.T @ θ).sum()

# Operational limits on the total number of items for the orders included in the wave:
n.addConstr(wave_size_ >= inst.LB * d)
n.addConstr(wave_size_ <= inst.UB * d)

# The selected corridors have sufficient storage for each of the items within the wave:
n.addConstrs(θ @ inst.u_oi[:, i] <= α @ inst.u_ai[:, i] for i in range(inst.I))

wave_corridors_ = α.sum()  # number of used corridors times d
n.addConstr(wave_corridors_ == 1)  # equivalent to d = 1/wave_corridors_

## Objective function:
n.setObjective(wave_size_, GRB.MAXIMIZE)

## Model solving:
dX = 1
try:
    n.optimize()

    # Print the found solution:
    dX = d.X
    print('\nBest solution found:')
    for v in n.getVars()[-10:]:  # limit the output
        # if v.VarName == 'd': continue
        print(f'{v.VarName} {v.X / dX:g}')
    print(f'Obj {wave_size_.getValue() / dX:g}')

    print(f'\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}')
    print(f'Wave size: {wave_size_.getValue():g}')
    print(f'Number of used corridors: {wave_corridors_.getValue() / dX:g}\n')
except gp.GurobiError as e:
    print(f'Error code {e.errno}: {e}')
except AttributeError:
    print('Encountered an attribute error')

Set parameter Username
Set parameter LicenseID to value 2615956
Academic license - for non-commercial use only - expires 2026-01-28
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6410 rows, 2787 columns and 32190 nonzeros
Model fingerprint: 0x60f76250
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [2e+00, 1e+02]
  Bounds range     [6e-03, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 19 rows and 26 columns
Presolve time: 0.02s
Presolved: 6391 rows, 2761 columns, 32058 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.02s

Barrier statistics:
 Dense cols : 113
 AA' NZ     : 6.849e+04
 Factor NZ  : 3.077e+05 (roughly 6 MB of me

# Full model

Solve using gurobi's MINLP formulation.

In [5]:
%%time
%%script echo skipping  # uncomment to skip
# Create a new model
m = gp.Model('meli-nonlinear')

## Variables:
O_ = m.addMVar(inst.O, vtype=GRB.BINARY, name='o')  # wave selected requests
A_ = m.addMVar(inst.A, vtype=GRB.BINARY, name='a')  # wave visited corridors
obj = m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=inst.u_oi.sum(), name='Obj')  # the objective cost function
m.update()

## Restrictions:
wave_size = (inst.u_oi.T @ O_).sum()  # total number of items in the wave

# Operational limits on the total number of items for the orders included in the wave:
m.addConstr(wave_size >= inst.LB)
m.addConstr(wave_size <= inst.UB)

# The selected corridors have sufficient storage for each of the items within the wave:
m.addConstrs(O_ @ inst.u_oi[:, i] <= A_ @ inst.u_ai[:, i] for i in range(inst.I))

m.addConstr(obj <= wave_size)  # basic cut
m.addConstr(obj <= wave_size_.getValue() / dX)  # linear relaxation UB

## Objective function:
wave_corridors = A_.sum()  # number of used corridors
m.addConstr(obj == wave_size / wave_corridors)
m.setObjective(obj, GRB.MAXIMIZE)

## Model solving:
try:
    m.optimize()

    # Print the found solution:
    print('\nBest solution found:')
    for v in m.getVars()[-10:]:  # limit the output
        print(f'{v.VarName} {v.X:g}')

    print(f'\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}')
    print(f'Wave size: {wave_size.getValue():g}')
    print(f'Number of used corridors: {wave_corridors.getValue():g}\n')
except gp.GurobiError as e:
    print(f'Error code {e.errno}: {e}')
except AttributeError:
    print('Encountered an attribute error')

skipping # uncomment to skip
CPU times: user 4.16 ms, sys: 11 μs, total: 4.17 ms
Wall time: 20.9 ms


# Binary search approach:

Perform a binary search on the objective function, running a linear-time decision variant of the model at each iteration.

In [6]:
%%time
%%script echo skipping  # uncomment to skip

# Create a new model
s = gp.Model('meli-search')

## Variables:
O_ = s.addMVar(inst.O, vtype=GRB.BINARY, name='o')  # wave selected requests
A_ = s.addMVar(inst.A, vtype=GRB.BINARY, name='a')  # wave visited corridors
s.update()

## Restrictions:
wave_size = (inst.u_oi.T @ O_).sum()  # total number of items in the wave
wave_corridors = A_.sum()  # number of used corridors

# Operational limits on the total number of items for the orders included in the wave:
s.addConstr(wave_size >= inst.LB)
s.addConstr(wave_size <= inst.UB)

# The selected corridors have sufficient storage for each of the items within the wave:
s.addConstrs(O_ @ inst.u_oi[:, i] <= A_ @ inst.u_ai[:, i] for i in range(inst.I))


def save_solution(model):
    solution = {var.VarName: var.X for var in model.getVars()}
    solution['Obj'] = wave_size.getValue() / wave_corridors.getValue()
    solution['Wave size:'] = wave_size.getValue()
    solution['Number of used corridors:'] = wave_corridors.getValue()
    return solution


## Model solving:
try:
    TOL = 1E-3  # GAP tolerance for convergence
    OPT_LB = 1  # starting LB
    OPT_UB = wave_size_.getValue() / dX  # starting UB from linear relaxation
    best_solution = {}
    best_obj = float('-inf')

    # First test if we can achieve the same cost of the linear relaxation:
    s.addConstr(wave_size >= OPT_UB * wave_corridors, name='LB')  # cost LB
    s.setObjective(wave_size, GRB.MAXIMIZE)  # fixed the LB maximize the wave size
    s.optimize()

    if s.status == GRB.OPTIMAL:
        best_solution = save_solution(s)
    else:
        # Cannot achieve the same cost of the linear relaxation, so do the search:
        s.setParam('LogToConsole', 0)  # Disable verbose output
        s.setObjective(0)  # transform the MIP into a decision problem
        print('\nStarting binary search...')
        while (OPT_UB - OPT_LB) / OPT_LB > TOL:
            middle = (OPT_UB + 2 * OPT_LB) / 3
            s.remove(s.getConstrByName('LB'))  # remove the old LB
            s.addConstr(wave_size >= middle * wave_corridors, name='LB')

            GAP = (OPT_UB - OPT_LB) / OPT_LB
            print(f'Current interval = [{OPT_LB:.3f}:{middle:.3f}:{OPT_UB:.3f}]; GAP = {100 * GAP:.3f}%')

            s.optimize()
            if s.status == GRB.INFEASIBLE:  # the LB is too high
                OPT_UB = middle
                continue

            # The LB is too low:
            current_obj = wave_size.getValue() / wave_corridors.getValue()
            OPT_LB = current_obj
            # Save the current solution if it is the new best one:
            if current_obj > best_obj: best_solution = save_solution(s)

    if len(best_solution) == 0:  # the model is infeasible
        raise Exception('No feasible solution found.')

    # Print the found solution:
    print('\nBest solution found:')
    for name, value in list(best_solution.items())[-12:-2]:
        print(f'{name} {value:g}')

    print(f'\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}')
    for name, value in list(best_solution.items())[-2:]:
        print(f'{name} {value:g}')
    print()
except gp.GurobiError as e:
    print(f'Error code {e.errno}: {e}')
except AttributeError:
    print('Encountered an attribute error')

skipping # uncomment to skip
CPU times: user 1.73 ms, sys: 827 μs, total: 2.55 ms
Wall time: 2.56 ms


# Binary search with Ortools:

In [ ]:
%%time
# %%script echo skipping  # uncomment to skip
# Create a new model
base = cp_model.CpModel()
O_vars = [base.NewBoolVar(f'o_{i}') for i in range(inst.O)]
A_vars = [base.NewBoolVar(f'a_{j}') for j in range(inst.A)]

# Precompute weights
u_oi = inst.u_oi
u_ai = inst.u_ai
item_weights = [int(u_oi[i].sum()) for i in range(inst.O)]

# Expressions (these reference the O_vars/A_vars objects)
total_items = sum(item_weights[i] * O_vars[i] for i in range(inst.O))
total_corridors = sum(A_vars)

# Static constraints
base.Add(total_items >= inst.LB)
base.Add(total_items <= inst.UB)
for i in range(inst.I):
    base.Add(
        sum(int(u_oi[o, i]) * O_vars[o] for o in range(inst.O))
        <=
        sum(int(u_ai[a, i]) * A_vars[a] for a in range(inst.A))
    )

# Grab the underlying proto so we can pop the ratio constraint later
proto = base.Proto()
base_n_cons = len(proto.constraints)

# 2) Prepare solver once
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 16
solver.parameters.log_search_progress = False

# 3) Binary search
TOL = 1e-2
scale = 100
OPT_LB = 1.0
OPT_UB = 251  # wave_size_.getValue() / dX  # starting UB from linear relaxation
best_obj = float('-inf')
best_sol = None

print("\nStarting binary search...")
while (OPT_UB - OPT_LB) / OPT_LB > TOL:
    mid = (OPT_UB + OPT_LB) / 2
    scaled_mid = int(mid * scale)

    # Add only the ratio constraint
    ratio_con = base.Add(total_items * scale >= scaled_mid * total_corridors)

    # Feasibility solve
    solver.Solve(base)

    if solver.StatusName() == 'INFEASIBLE':
        OPT_UB = mid
    else:
        # Read out the solution
        val_items = sum(item_weights[i] * solver.Value(O_vars[i]) for i in range(inst.O))
        val_corrs = sum(solver.Value(a) for a in A_vars)
        if val_corrs > 0:
            curr_obj = val_items / val_corrs
            if curr_obj > best_obj:
                best_obj = curr_obj
                best_sol = { f'o_{i}': solver.Value(O_vars[i]) for i in range(inst.O) }
                best_sol.update({ f'a_{j}': solver.Value(A_vars[j]) for j in range(inst.A) })
                best_sol['Obj'] = curr_obj
                best_sol['Wave size'] = val_items
                best_sol['Number of used corridors'] = val_corrs
            OPT_LB = curr_obj
        else:
            # no corridors ⇒ ratio undefined ⇒ raise lower bound
            OPT_LB = mid

    # **Remove** the ratio constraint we just added
    # (we know it's the last one in proto.constraints)
    del proto.constraints[-1]

    gap = (OPT_UB - OPT_LB) / OPT_LB
    print(f'Interval = [{OPT_LB:.3f}:{mid:.3f}:{OPT_UB:.3f}] GAP = {100*gap:.2f}%')

# 4) Check & print
if best_sol is None:
    raise Exception("No feasible solution found.")

print("\nBest solution found:")
for k, v in list(best_sol.items())[-10:]:
    print(f"{k}: {v}")
print(f"\nRequests, Items, Corridors = {inst.O}, {inst.I}, {inst.A}")